sentiment analysis

In [14]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Embedding , LSTM
from sklearn.model_selection import train_test_split
import re

In [15]:
df=pd.read_csv('Sentiment.csv')

In [16]:
df.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [17]:
data=df[['text','sentiment']]

In [18]:
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [19]:
print(data['sentiment'].unique())

['Neutral' 'Positive' 'Negative']


In [20]:
data['sentiment']=data['sentiment'].str.strip().str.lower()

<ipython-input-20-229a2e7238e2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment']=data['sentiment'].str.strip().str.lower()


In [21]:
data=data[data['sentiment']!= 'neutral']

In [22]:
data['text']=data['text'].apply(lambda x: x.lower())
data['text']=data['text'].apply((lambda x: re.sub('(#|@)\w*','',x))) # remove @username
data['text']=data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['text']=data['text'].apply((lambda x: re.sub('^rt ',' ',x)))






In [23]:
data['text'].head()

1      didnt catch the full  last night here are so...
3      that carly fiorina is trending  hours after ...
4       w  delivered the highest ratings in the his...
5       on my first day i will rescind every illega...
6      i liked her and was happy when i heard she w...
Name: text, dtype: object

#tokenization

In [30]:
max_features=2500   # will make dict and sort it tanazoly according to no. of repeations of the word ... the more repeated will be first
tokenizer=Tokenizer(num_words=max_features,split=' ') #oov_token = '<unk>' عشان لو كلمة مش فالقاموس يحطها ف الother
tokenizer.fit_on_texts(data['text'].values)  # .values to convert to array
X=tokenizer.texts_to_sequences(data['text'].values)

In [31]:
X[0]

[114, 2079, 1, 646, 35, 51, 221, 32, 192, 4, 162, 1549, 10, 1163, 1242]

In [32]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'is': 3,
 'of': 4,
 'a': 5,
 'and': 6,
 'i': 7,
 'you': 8,
 'it': 9,
 'in': 10,
 'for': 11,
 'trump': 12,
 'that': 13,
 'on': 14,
 'fox': 15,
 'not': 16,
 'this': 17,
 'was': 18,
 'about': 19,
 'amp': 20,
 'news': 21,
 'debate': 22,
 'from': 23,
 'have': 24,
 'at': 25,
 'we': 26,
 'me': 27,
 'but': 28,
 'he': 29,
 'be': 30,
 'candidates': 31,
 'are': 32,
 'just': 33,
 'like': 34,
 'last': 35,
 'they': 36,
 'so': 37,
 'dont': 38,
 'with': 39,
 'all': 40,
 'up': 41,
 'what': 42,
 'people': 43,
 'who': 44,
 'jeb': 45,
 'my': 46,
 'how': 47,
 'as': 48,
 'if': 49,
 'bush': 50,
 'night': 51,
 'has': 52,
 'no': 53,
 'their': 54,
 'by': 55,
 'when': 56,
 'can': 57,
 'do': 58,
 'one': 59,
 'more': 60,
 'im': 61,
 'or': 62,
 'out': 63,
 'would': 64,
 'his': 65,
 'him': 66,
 'chris': 67,
 'get': 68,
 'think': 69,
 'gop': 70,
 'donald': 71,
 'really': 72,
 'president': 73,
 'republican': 74,
 'its': 75,
 'will': 76,
 'why': 77,
 'want': 78,
 'did': 79,
 'ask': 80,
 'need': 81

In [34]:
print(X[0])
print(X[1])

[114, 2079, 1, 646, 35, 51, 221, 32, 192, 4, 162, 1549, 10, 1163, 1242]
[13, 267, 299, 3, 736, 96, 158, 22, 2395, 127, 4, 1, 161, 10, 215, 647, 14]


In [38]:
X=pad_sequences(X)

In [39]:
X.shape

(10729, 26)

In [40]:
print(X[0])
print(X[1])

[   0    0    0    0    0    0    0    0    0    0    0  114 2079    1
  646   35   51  221   32  192    4  162 1549   10 1163 1242]
[   0    0    0    0    0    0    0    0    0   13  267  299    3  736
   96  158   22 2395  127    4    1  161   10  215  647   14]


In [41]:
embed_dim=128
model=Sequential()
model.add(Embedding(max_features,embed_dim,input_length=X.shape[1]))#X.shape[1]=26
model.add(LSTM(196))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [42]:
data['sentiment']=data['sentiment'].replace({'positive':1,'negative':0})

In [44]:
data

,text,sentiment
1,didnt catch the full last night here are so...,1
3,that carly fiorina is trending hours after ...,1
4,w delivered the highest ratings in the his...,1
5,on my first day i will rescind every illega...,1
6,i liked her and was happy when i heard she w...,0
...,...,...
13866,love to see men who will never be faced with...,0
13867,who thought huckabee exceeded their expectat...,1
13868,as president i will always tell the truth a...,1
13869,donald trump says that he doesnt have time ...,0


In [45]:
x_train,x_test,y_train,y_test=train_test_split(X,data['sentiment'],test_size=0.2,random_state=42)

In [46]:
model.fit(x_train,y_train,epochs=10,batch_size=32)

Epoch 1/10
269/269 [==============================] - 27s 89ms/step - loss: 0.4273 - accuracy: 0.8231
Epoch 2/10
269/269 [==============================] - 22s 83ms/step - loss: 0.3043 - accuracy: 0.8731
Epoch 3/10
269/269 [==============================] - 25s 94ms/step - loss: 0.2540 - accuracy: 0.8935
Epoch 4/10
269/269 [==============================] - 24s 88ms/step - loss: 0.2074 - accuracy: 0.9162
Epoch 5/10
269/269 [==============================] - 21s 78ms/step - loss: 0.1767 - accuracy: 0.9281
Epoch 6/10
269/269 [==============================] - 23s 84ms/step - loss: 0.1484 - accuracy: 0.9384
Epoch 7/10
269/269 [==============================] - 21s 78ms/step - loss: 0.1288 - accuracy: 0.9482
Epoch 8/10
269/269 [==============================] - 22s 83ms/step - loss: 0.1160 - accuracy: 0.9534
Epoch 9/10
269/269 [==============================] - 20s 75ms/step - loss: 0.1014 - accuracy: 0.9581
Epoch 10/10
269/269 [==============================] - 22s 82ms/step - loss: 0.098